In [9]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [14]:
feature_store = get_feature_store()
print("✅ Feature Store Connected")


2025-03-05 23:49:00,202 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 23:49:00,205 INFO: Initializing external client
2025-03-05 23:49:00,205 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 23:49:01,851 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
✅ Feature Store Connected


In [16]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1))
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-05 23:50:36,735 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 23:50:36,738 INFO: Initializing external client
2025-03-05 23:50:36,738 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 23:50:37,903 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Fetching data from 2025-02-05 04:50:36.735022+00:00 to 2025-03-06 03:50:36.735022+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.28s) 


In [17]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 23:50:53,674 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 23:50:53,675 INFO: Initializing external client
2025-03-05 23:50:53,676 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 23:50:54,728 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676

In [18]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,3,0,1,0,1,0,3,0,0,...,0,4,0,2,1,0,3,4,188,2025-03-05 06:00:00
1,1,1,0,0,1,1,2,1,2,0,...,0,0,0,0,0,0,0,1,196,2025-03-05 06:00:00
2,32,136,124,24,123,145,229,57,130,85,...,57,25,106,41,132,125,21,124,263,2025-03-05 06:00:00
3,2,0,1,3,0,0,24,0,0,0,...,2,0,0,1,3,0,2,1,255,2025-03-05 06:00:00
4,0,0,0,1,0,0,0,1,1,0,...,8,0,0,1,0,1,0,0,112,2025-03-05 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,122,2025-03-05 06:00:00
248,67,70,69,81,1,75,122,165,22,93,...,60,124,121,74,124,4,70,55,231,2025-03-05 05:00:00
249,0,0,0,0,0,1,0,0,0,1,...,2,0,0,2,2,0,2,0,205,2025-03-05 05:00:00
250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,2025-03-05 05:00:00


In [19]:
model

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x13d62d620>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [20]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [21]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,188,2.0,2025-03-06 05:00:00+00:00
1,196,0.0,2025-03-06 05:00:00+00:00
2,263,117.0,2025-03-06 05:00:00+00:00
3,255,3.0,2025-03-06 05:00:00+00:00
4,112,2.0,2025-03-06 05:00:00+00:00
...,...,...,...
247,122,0.0,2025-03-06 05:00:00+00:00
248,231,69.0,2025-03-06 05:00:00+00:00
249,205,0.0,2025-03-06 05:00:00+00:00
250,30,0.0,2025-03-06 05:00:00+00:00


In [22]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-05 23:51:34,633 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 23:51:34,636 INFO: Initializing external client
2025-03-05 23:51:34,636 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 23:51:35,797 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676


In [23]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1215676/fs/1203303/fg/1403591


Uploading Dataframe: 100.00% |█████████████████████████| Rows 252/252 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215676/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)